<div class="alert alert-block alert-info">
<h3>Student Information</h3> Please provide information about yourself. We will NOT grade this submissing w/o all the information<br>
<b>Name</b>: Ethan Wang<br>
<b>NetID</b>: ew337<br>
<b>Recitation (01,02,03)</b>: 3<br>
<b>Notes to Grader</b> (optional): For 3.11a and 3.11b, I'm fairly certain that the expected output for 3.11a and 3.11b is wrong. Most of the values are above 1, so they can't be probabilities. <br>
<br><br>
<b>IMPORTANT</b>
Your work will not be graded withour your initials below<br>
I certify that this lab represents my own work and I have read the RU academic intergrity policies at<br>
<a href="https://www.cs.rutgers.edu/academic-integrity/introduction">https://www.cs.rutgers.edu/academic-integrity/introduction </a><br>
<b>Initials</b>: EW      


<h3>Grader Notes</h3>
<b>Your Grade<b>:<br>
<b>Grader Initials</b>:<br>
<b>Grader Comments</b> (optional):<br>
</div>

## CS 439 - Introduction to Data Science
### Spring 2022

# Lab 3: Exploratary Data Analysis

### Due Date: Thursday March 03, 2022 by 11:59 PM ###

### Instructions
This lab is presented as a notebook. Please execute the cells that are already completed and your task is to fill in the code
between ### BEGIN SOLUTION ### and ### END SOLUTION ###. 

#### Important: Please do not add any new cells or change the order of cells. If you have questions, please contact the courseS staff.

In this lab, you will be working with a dataset from NYPD containing data on calls to the New York Police Department. Information about the datasets can be found at https://opendata.cityofnewyork.us/

## Setup

Note that after activating matplotlib to display figures inline via the IPython magic `%matplotlib inline`, we configure a custom default figure size. Virtually every default aspect of matplotlib [can be customized](https://matplotlib.org/users/customizing.html).

In [ ]:
import pandas as pd
import numpy as np
import zipfile
import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline
plt.rcParams['figure.figsize'] = (12, 9)

# Part 1:Getting Data

We will work with the NYPD Historic complaint data set. Our first task is to estimate the size of this download by looking at the number of rows, columns and using an estimated size for a column (use a reasonable value). The site metadata is available from the page
https://data.cityofnewyork.us/Public-Safety/NYPD-Complaint-Data-Historic/qgea-i56i

In [ ]:
### BEGIN SOLUTION
estimated size of the download (based on metadata information from the site) = 2066400000 #size in bytes 

# please explain how you reached the answer

    # 7380000 * 35 * 8 = 2066400000 bytes
    # We assume that each entry in the dataset is an object or int64/float64 variable of 8 bytes
    
### END SOLUTION

### 1.1 Download the data
This file is large (use the estimate you did above). If it takes too long to download, you may want to interrupt and download the file using a browser and URL https://data.cityofnewyork.us/Public-Safety/NYPD-Complaint-Data-Historic/qgea-i56i

In [ ]:
import utils
data_dir = 'data'
data_url = 'https://data.cityofnewyork.us/Public-Safety/NYPD-Complaint-Data-Historic/qgea-i56i'

file_name = 'NYPD_Complaint_Data_Historic.csv'

# To retrieve the dataset, we will use the `utils.fetch_and_cache` utility from utils library. 
dest_path = utils.fetch_and_cache(data_url=data_url, file=file_name,data_dir=data_dir)
print(f'Located at {dest_path}')

### 1.2 Inspect the size of the file
It is helpful to get an idea of the size of the file. This can be done using functions in the utils library.

In [ ]:
# look at the size of the file w/o opening it using OS (https://docs.python.org/3/library/os.html). You can perform
# variety of operating system related functions from this package.
### BEGIN SOLUTION
import os

size = os.path.getsize('data/NYPD_Complaint_Data_Historic.csv')
size
#### END SOLUTION

### 1.3 Split the large file
This data file NYPD_Complaint_Data_Historic.csv is too big to load into a single DataFrame. Let us split the large file into smaller files.  Let us find out the number of lines in the NYPD_Complaint_Data_Historic.csv file using utils.

In [ ]:
# Using utils library, find the number of lines in the file

### BEGIN SOLUTION
import utils
utils.line_count('data/NYPD_Complaint_Data_Historic.csv')

### END SOLUTION

In [ ]:
# Split the file into 10 smaller files. Estimate the number of lines in each file using the count above
# files should be created in the data folder and named NYPD_Complaint_Data_Historic_1.csv, 
# NYPD_Complaint_Data_Historic_2.csv, ... NYPD_Complaint_Data_Historic_10.csv etc
# It is possible that few lines from the original file may not be saved due to rounding errors.

### BEGIN SOLUTION
import math

csvfile = open('data/NYPD_Complaint_Data_Historic.csv', 'r').readlines()
filename = 1
for i in range(len(csvfile)):
    if i % 737600 == 0:
        open('data/NYPD_Complaint_Data_Historic_' + str(filename) + '.csv', 'w+').writelines(csvfile[i:i+737600])
        filename += 1

math.ceil(utils.line_count('data/NYPD_Complaint_Data_Historic.csv') / 10) #estimated number of lines in each file
### END SOLUTION

### 1.4 Check the files in the data folder
Now, we'll use a method of the `Pathlib.Path` class called `glob` to list all files in the `data` directory. You will find useful information in pathlib [docs](https://docs.python.org/3/library/pathlib.html).

Below, we use pathlib's `glob` method to store the list of all files' names from the `data_dir` directory in the variable `file_names`. These names should be strings that contain only the file name (e.g. `dummy.txt` not `data/dummy.txt`). The asterisk (*) character is used with the `glob` method to match any string.

In [ ]:
from pathlib import Path
data_dir_path = Path('data') # creates a Path object that points to the data directory
file_names = [x.name for x in data_dir_path.glob('*') if x.is_file()]
file_names

### 1.5 Pre-processing of data
It is good to pre-process the data to see if the file can be opened in a Jupyter notebook. We need to avoid large files that can crash notebooks. Typically, files of size around 200 MB is ok to open into a DataFrame. In the following activities we will inspect the file w/o opening it as a DataFrame. Using utils.head 

In [ ]:
# Use the util.head() to read up to 5 lines from the original file (w/o opening it)
from utils import head
head('data/NYPD_Complaint_Data_Historic.csv')

### 1.6 Inspecting and describing data columns
There should be 35 columns in each record. Using header information and data types, describe the type of data in each
column. If you are unable to determine, just state so.
##### BEGIN SOLUTION
* CMPLNT_NUM : An integer
* CMPLNT_FR_DT: A string
* CMPLNT_FR_TM: A string
* CMPLNT_TO_DT: A string
* CMPLNT_TO_TM: A string
* ADDR_PCT_CD: An integer
* RPT_DT: A string
* KY_CD: A integer
* OFNS_DESC: A string
* PD_CD: A integer
* PD_DESC: A string
* CRM_ATPT_CPTD_CD: A string
* LAW_CAT_CD: A string
* BORO_NM: A string
* LOC_OF_OCCUR_DESC: A string
* PREM_TYP_DESC: A string
* JURIS_DESC: A string
* JURISDICTION_CODE: An integer
* PARKS_NM: A string
* HADEVELOPT: A string
* HOUSING_PSA: An integer
* X_COORD_CD: A float
* Y_COORD_CD: A float
* SUSP_AGE_GROUP: A string
* SUSP_RACE: A string
* SUSP_SEX: A string
* TRANSIT_DISTRICT: A string
* Latitude: A float
* Longitude: A float
* Lat_Lon: A string
* PATROL_BORO: A string
* STATION_NAME: A string
* VIC_AGE_GROUP: A string
* VIC_RACE: A string
* VIC_SEX: A string
##### END SOLUTION

# Part 2 - Exploratory Data Analysis
Exploratory data analysis (EDA) is the process of examining a subest of a large data set to see what we can know about the data. First we will explore one file NYPD_Complaint_Data_Historic_1.csv to see what we can find out.

### 2.1 Loading Data into a DataFrame
Load the first CSV file, NYPD_Complaint_Data_Historic_1.csv into a `pandas.DataFrame` object. Also do a time analysis to see how long it took to load the data into a DataFrame. Time should be printed in seconds. The time libraries https://docs.python.org/3/library/time.html can help.

In [ ]:
### BEGIN SOLUTION
import pandas as pd
import time

start = time.time()
calls = pd.read_csv("data/NYPD_Complaint_Data_Historic_1.csv")
end = time.time()

print(end - start)
### END SOLUTION

### 2.2 Description of Fields
Let's also check some basic information about these files using the `DataFrame.describe` and `DataFrame.info` methods. Describe columns that can be removed based on the information.

In [ ]:
### BEGIN SOLUTION
calls.info()
calls.describe()

# What columns can be removed from the DataFrame? A reasonable rule of thumb is that if a column is missing more than 
# 50% of the data then it should be removed

# Columns PARKS_NM, HADEVELOPT, HOUSING_PSA, TRANSIT_DISTRICT, and STATION_NAME should be removed since they don't
# even have more than 100,000 non-null objects

### END SOLUTION

### 2.2 Finding Uniques

Notice that the functions above reveal type information for the columns, as well as some basic statistics about the numerical columns found in the DataFrame. However, we still need more information about what each column represents. Let's explore the data further.

find the number of unique values in each DataFrame column and answer the questions below.

In [ ]:
### BEGIN SOLUTION
calls.nunique()

# Questions
# 1. How many distinct locations where the complaints have come from?

    # 5 distinct locations

# 2. How many age groups are represented in the data set?

    # There appear to be 25 age groups in SUSP_AGE_GROUP, and 44 age groups in VIC_AGE_GROUP. However, if we analyze
    # those columns more, we can conclude that there are only 5 distinct age-groups that are thoroughly represented
    # by those columns. The rest are either outliers or unknown. Therefore, there are 5 age groups being represented
    # in the data set.

# 3. How many boroughs are included in the data set?

    # There are 5 distinct boroughs

# 4. How many offense types are listed in this data set?

    # There are 65 different offense types

### END SOLUTION

### 2.3 Offense by Boro
Using GroupBy operation, create a DataFrame that groups offenses by Boro. call the DataFrame calls_by_Boro_and_offense

In [ ]:
### BEGIN SOLUTION
calls_by_Boro_and_offense = calls.groupby(["BORO_NM", "OFNS_DESC"], dropna=False).count()
calls_by_Boro_and_offense
### END SOLUTION

### 2.4 Offenses in Bronx

In the cell below, find a list of strings corresponding to the possible values for `OFNS_DESC` when `BORO` is "BRONX". Create an expression that automatically extracts the names of the offenses.

In [ ]:
### BEGIN SOLUTION
print(list(calls_by_Boro_and_offense.loc['BRONX'].index))
### END SOLUTION

# How many offenses were committed in Bronx during the analysis period?
calls_by_Boro_and_offense.loc['BRONX']['CMPLNT_NUM'].sum()

### 2.5 Most Common Crimes in NYC

What are the five crime types of OFNS_DESC that have the most crime events in Bronx? You may need to use `value_counts` to find the answer. Save your results as a list of strings.

**Hint:** *The `keys` method of the Series class might be useful.*

In [ ]:
### BEGIN SOLUTION
common = calls_by_Boro_and_offense.loc['BRONX']['CMPLNT_NUM'].sort_values(ascending=False).keys()[0:5]
print(list(common))
### END SOLUTION

### 2.6 Probability of a Crime in Bronx
What is the probability that a the crime "Arson" can happen in Bronx?

In [ ]:
### BEGIN SOLUTION
arson = calls_by_Boro_and_offense.loc['BRONX', 'ARSON']['CMPLNT_NUM']
total = calls_by_Boro_and_offense.loc['BRONX']['CMPLNT_NUM'].sum()

prob = arson/total
prob
### END SOLUTION

---
# Part 3: Visualizing the Data

## Pandas vs. Seaborn Plotting

Pandas offers basic functionality for plotting. For example, the `DataFrame` and `Series` classes both have a `plot` method. However, the basic plots generated by pandas are not particularly pretty. While it's possible to manually use matplotlib commands to make pandas plots look better, we'll instead use a high level plotting library called Seaborn that will take care of most of this for us.

As you learn to do data visualization, you may find the [pandas documentation](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.plot.html) and [Seaborn documentation](https://seaborn.pydata.org/api.html) helpful!

### 3.1 Plotting a Series
Using the built-in plotting functionality of pandas, such as `plot` method of the `Series` class to generate a `barh` plot type,  display the value counts for `OFNS_DESC` visually as a barh chart.

In [ ]:
### BEGIN SOLUTION

xlab = list(calls_by_Boro_and_offense.loc['BRONX']['CMPLNT_NUM'].sort_values(ascending=False))
ylab = list(calls_by_Boro_and_offense.loc['BRONX']['CMPLNT_NUM'].sort_values(ascending=False).index)

ofns = pd.DataFrame({'Crime Category': ylab, 'Number of Calls': xlab})
ofnsplot = ofns.plot.barh(x='Crime Category', y='Number of Calls')
ofnsplot.set_xlabel('Number of Calls')
ofnsplot.set_title('Number of Calls By Crime Type');

### END SOLUTION

### 3.2 Getting a Better Plot
The plot above can be messy as it plots all offenses. Plot only the offenses that has more than 10000 calls

In [ ]:
### BEGIN SOLUTION
ten = calls_by_Boro_and_offense[calls_by_Boro_and_offense['CMPLNT_NUM'] > 10000]

xten = list(ten.loc['BRONX']['CMPLNT_NUM'].sort_values(ascending=False))
yten = list(ten.loc['BRONX']['CMPLNT_NUM'].sort_values(ascending=False).index)

ofnsten = pd.DataFrame({'Crime Category': yten, 'Number of Calls': xten})
tenplot = ofnsten.plot.barh(x='Crime Category', y='Number of Calls')

tenplot.set_xlabel("Number of Calls")
tenplot.set_title("Number of Calls By Crime Type");
### END SOLUTION

By contrast, the Seaborn library provides a specific function `countplot` built for plotting counts. It operates directly on the DataFrame itself i.e. there's no need to call `value_counts()` at all. This higher level approach makes it easier to work with. Use the y-label ("Crime Category"), x-label("Number of Calls") and title_of_plot("Number of Calls By Crime Type"

In [ ]:
### BEGIN SOLUTION
seabarplot = sns.countplot(y = 'OFNS_DESC', data=calls)

seabarplot.set(xlabel='Number of Calls', ylabel='Crime Category', title='Number of Calls By Crime Type');
### END SOLUTION

You may note that the ordering might be different for the seaborn plot (as compared to pandas plot). If we want the same ordering that we had in the pandas plot, we can use the order parameter of the `countplot` method. It takes a list of strings corresponding to the axis to be ordered. By passing the index of the `value_counts`, you can get the order you want.

In [ ]:
### BEGIN SOLUTION
orderplot = sns.countplot(y='OFNS_DESC', order=calls['OFNS_DESC'].value_counts().sort_values(ascending=True).index, data=calls)

orderplot.set(xlabel='Number of Calls', ylabel='Crime Category', title='Number of Calls By Crime Type');
### END SOLUTION

Now we have a pretty bar plot with the bars ordered by size. Though `seaborn` appears to provide a superior plot from a aesthetic point of view, the `pandas` plotting library is also good to understand. You'll get practice using both libraries in the following questions.

## An Additional Note on Plotting in Jupyter Notebooks

You may have noticed that many of our code cells involving plotting end with a semicolon (;). This prevents any extra output from the last line of the cell that we may not want to see. Try adding this to your own code in the following questions!

### 3.3 making more plots

Now it is your turn to make some more plots using `pandas` and `seaborn`. Let's start by looking at the distribution of calls over days of the week.

The CMPLNT_FR_DT field contains the date of the event. We would like to add a new column to the DataFrame that includes Day of the week (DAY_OF_WEEK) that indicates the day of the week. This can help us analyze the crimes on a specific day of the week. For example, we can answer questions such as "what day of the week that a LARSON is likely to happen in NYC?"


Add a new column `DAY_OF_WEEK` into the `calls` dataframe that has the day string (eg. 'Sunday') for the corresponding value in CMPLNT_FR_DT. For example, if the first 3 values of `CMPLNT_FR_DT` are `['01/27/2006, '01/28/2006, '01/29/2006]`, then the first 3 values of the `DAY_OF_WEEK` column should be `["Friday", "Saturday", "Sunday"]`.

**Hint:** *Try using the [Series.map](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.Series.map.html) function on `calls["OFNS_DESC"]`.  Can you assign this to the new column `calls["DAY_OF_WEEK"]`?*

In [ ]:
days = ["Sunday", "Monday", "Tuesday", "Wednesday", "Thursday", "Friday", "Saturday"]
### BEGIN SOLUTION
calls["DAY_OF_WEEK"] = pd.to_datetime(calls["CMPLNT_FR_DT"], errors='coerce').dt.day_name()
calls["DAY_OF_WEEK"][0:13]

### END SOLUTION

### 3.4 Seaborn plots

Create a `seaborn` plot that shows the number of calls for each day of the week. You may want to use of the `rotation` argument in `ax.set_xticklabels`, which rotates the labels by 90 degrees.

In [ ]:
### BEGIN SOLUTION
seadaysplot = sns.countplot(x= 'DAY_OF_WEEK', order=days, data=calls)

seadaysplot.set(ylabel='Number of Calls', title='Number of Calls For Each Day of the Week')
seadaysplot.set_xticklabels(days, rotation=90);
### END SOLUTION

Now, let's make the same plot using `pandas`. Construct a vertical bar plot with the count of the number of calls (entries in the table) for each day of the week **ordered by the day of the week** (eg. `Sunday`, `Monday`, ...). Do not use `sns` for this plot. Be sure that your axes are labeled and that your plot is titled.

**Hint:** *Given a series `s`, and an array `coolIndex` that has the same entries as in `s.index`, `s[coolIndex]` will return a copy of the series in the same order as `coolIndex`.*

In [ ]:
### BEGIN SOLUTION
xdays = list(calls['DAY_OF_WEEK'].value_counts()[days])

dayscalls = pd.DataFrame({'Number of Calls': xdays, 'Day': days,})
daysplot = dayscalls.plot.bar(x='Day', y='Number of Calls')

daysplot.set_xticklabels(days, rotation=90)
daysplot.set_ylabel("Number of Calls")
daysplot.set_title("Number of Calls For Each Day of the Week");

### END SOLUTION

### 3.5 What Day of the Week is more calls?

Is it true that weekdays generally have slightly more calls than Saturday or Sunday? What can you say about the difference?

##### BEGIN SOLUTION
Based on the plot, every other weekday has more calls that Sunday's, however Monday's calls are smaller than Saturday's, and Thursday's calls appears to be tied with Saturday's. The reason why Sunday has the lowest amount of calls is probably because it's the day before the weekdays where most people go to work. Also the reason why Saturday is higher than some days is because a lot of people probably party and end up creating incidents around that time. It also explains why Friday has the highest amount of calls.

##### END SOLUTION

We can break down into some particular types of events to see their distribution. For example, let's make a bar plot for the OFNS_DESC "HARRASSMENT 2". Which day is the peak for "HARRASSMENT 2"?

This time, use `seaborn` to create a vertical bar plot of the number of total noise violations reported on each day of the week, again ordered by the days of the week starting with Sunday. Do not use `pandas` to plot.

**Hint:** *If you're stuck, use the code for the seaborn plot in above question as a starting point.*

In [ ]:
### BEGIN SOLUTION
harassplot = sns.countplot(x= 'DAY_OF_WEEK', order=days, data=calls[calls['OFNS_DESC'] == 'HARRASSMENT 2'])

harassplot.set_xticklabels(days, rotation=90)
harassplot.set(ylabel='Number of Calls', title='Number of Harrassment-2 Calls For Each Day of the Week');
### END SOLUTION

### 3.6 Distribution

Do you see anything interesting about the distribution of HARRASSMENT 2 calls over a week? Type a short answer below.
##### BEGIN SOLUTION
The number of calls on the weekdays are bigger than the number of calls on the weekends. This is probably because harassment often occurs in work settings like an office, and most people often stay home on the weekends.
##### END SOLUTION

### 3.7 More Plots

Let's look at a similar distribution but for a crime we have much more calls data about. In the cell below, create the same plot as you did in previous questions, but now looking at instances of the OFNS_DESC "BURGLARY" (instead of "HARRASSMENT 2"). Use either `pandas` or `seaborn` plotting as you desire.

In [ ]:
### BEGIN SOLUTION
ofnsburgplot = sns.countplot(x= 'DAY_OF_WEEK', order=days, data=calls[calls['OFNS_DESC'] == 'BURGLARY'])

ofnsburgplot.set_xticklabels(days, rotation=90)
ofnsburgplot.set(ylabel='Number of Calls', title='Number of BURGLARY Calls For Each Day of the Week');
### END SOLUTION



### 3.8 time of events

Now let's look at the CMPLNT_TO_TM column which indicates the time for events. Since it contains hour and minute information, let's extract the hour info and create a new column named `Hour` in the `calls` dataframe. You should save the hour as an `int`. Then plot the frequency of each hour in the table (i.e., `value_counts()`) sorted by the hour of the day (i.e., `sort_index()`).

You will want to look into how to use:

* [Series.str.slice](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.Series.str.slice.html#pandas.Series.str.slice) to select the substring.
* [Series.astype](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.Series.astype.html) to change the type.

**Hint:** *The `str` helper member of a series can be used to grab substrings.  For example, `calls["CMPLNT_TO_TM"].str.slice(3,5)` returns the minute of each hour of the `CMPLNT_TO_TM`.*

In [ ]:
### BEGIN SOLUTION
calls["Hour"] = calls["CMPLNT_TO_TM"].str.slice(0,2)
calls['Hour'] = calls['Hour'].fillna(0)
calls['Hour'] = calls['Hour'].astype('int64')
calls['Hour'].value_counts().sort_index()


xhour = list(calls['Hour'].value_counts().sort_index())
hours = calls['Hour'].value_counts().sort_index().index

hourcalls = pd.DataFrame({'Number of Calls': xhour, 'Hour': hours})
hourplot = hourcalls.plot.bar(x='Hour', y='Number of Calls')

hourplot.set_xlabel("Hour")
hourplot.set_ylabel("Number of Calls")
hourplot.set_title("Number of Calls Per Hour");

### END SOLUTION

Create a pandas bar plot showing the number of BURGLARY crimes committed at each hour of the day. Use the labels
* ax.set_xlabel("Hour of the Day")
* ax.set_ylabel("Number of Calls")
* ax.set_title("Number of Calls Reporting Fraud For Each Day of the Week");

In [ ]:
### BEGIN SOLUTION
burglary = calls[calls['OFNS_DESC'] == 'BURGLARY']

xburglary = list(burglary['Hour'].value_counts().sort_index())

burglarycalls = pd.DataFrame({'Number of Calls': xburglary, 'Hour': hours})
burghourplot = burglarycalls.plot.bar(x='Hour', y='Number of Calls')

burghourplot.set_xlabel("Hour of the Day")
burghourplot.set_ylabel("Number of Calls")
burghourplot.set_title("Number of Calls Reporting Fraud For Each Day of the Week");
### END SOLUTION

### 3.9 More plots

In the cell below, create a seaborn plot of the same data. Again, make sure you provide axes labels and a title for your plot.

In [ ]:
### BEGIN SOLUTION
burgplot = sns.countplot(x= 'Hour',  data=calls[calls['OFNS_DESC'] == 'BURGLARY'])
burgplot.set(xlabel='Hour of the Day', ylabel='Number of Calls', title='Number of Calls Reporting Fraud For Each Day of the Week');

### END SOLUTION

### 3.10 Spike in burglary?

According to your plots, is there a spike in calls reporting BURGLARY at any particular time? If so, Do you trust that this spike is legitimate, or could there be an issue with our data? Explain your reasoning in 1-2 sentences below.

#### BEGIN SOLUTION
Our plots show a spike at hour 0 and hour 7. Hour 0 is illegitimate since its number of calls also include all the NaN values filled in by zeroes. Hour 7 on the other hand, is a legitimate spike in the data. The reason why hour 7 seems reasonable is that most burglars often commit robberies in the middle of the night, like around 2:00 AM (Hour 2). So hour 7, or 7:00 AM is the appropriate time for when a person wakes up, sees that they got robbed, and calls the police.

#### END SOLUTION

In the cell below, we generate a boxplot which examines the hour of day of each crime broken down by the `OFNS_DESC` value.  To construct this plot we used the [DataFrame.boxplot](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.boxplot.html) documentation.

In [ ]:
calls.boxplot(column="Hour", by='OFNS_DESC', rot=90);

While the pandas boxplot is informative, we can use seaborn to create a more visually-appealing plot. Using seaborn, regenerate a better box plot. See either the textbook (https://www.textbook.ds100.org/ch/06/viz_quantitative.html) or the [seaborn boxplot documentation](https://seaborn.pydata.org/generated/seaborn.boxplot.html).

Looking at your plot, which crime type appears to have the largest interquartile range? Put your results into `answer` as a string.

In [ ]:
# Todo: Make a boxplot with seaborn
### BEGIN SOLUTION
ax = sns.boxplot(x='OFNS_DESC', y='Hour', data=calls)
ax.set_xticklabels(ax.get_xticklabels(),rotation=90)
ax.set_title("Distributions of Calls Over Hours in a Day, Grouped by Crime");
answer = "PROSTITUTION & RELATED OFFENSES"
### END SOLUTION

### 3.11a Explore Suspect and Victim Age Relationships
In this activity we explore relations between SUSP_AGE_GROUP and VIC_AGE_GROUP
1. create a matrix of where rows are SUSP_AGE_GROUPS and Columns are VIC_AGE_GROUPs
2. Fill in the matrix with all probabilities (eg. "what is the probability that <18 susp_group target <18 victim_group)

In [ ]:
### BEGIN SOLUTION
sus = calls[(calls['SUSP_AGE_GROUP'] == '25-44') | (calls['SUSP_AGE_GROUP'] == '<18') | (calls['SUSP_AGE_GROUP'] == '45-64') |
     (calls['SUSP_AGE_GROUP'] == '18-24') | (calls['SUSP_AGE_GROUP'] == '65+')]

vic = sus[(sus['VIC_AGE_GROUP'] == '25-44') | (sus['VIC_AGE_GROUP'] == '<18') | (sus['VIC_AGE_GROUP'] == '45-64') |
     (sus['VIC_AGE_GROUP'] == '18-24') | (sus['VIC_AGE_GROUP'] == '65+')]

viccount = pd.crosstab(vic.SUSP_AGE_GROUP, vic.VIC_AGE_GROUP) #count matrix

vicprobs = pd.crosstab(vic.SUSP_AGE_GROUP, vic.VIC_AGE_GROUP) / len(vic) #frequency matrix
vicprobs

### END SOLUTION

### 3.11b Explore Suspect and Victim Sex Relationships
In this activity we explore relations between SUSP_SEX and VIC_SEX 
1. create a matrix of where rows are SUSP_SEX and Columns are VIC_SEX
2. Fill in the matrix with all probabilities (eg. "what is the probability that Males target other Males?")

In [ ]:
### BEGIN SOLUTION
susp = calls[(calls['SUSP_SEX'] == 'M') | (calls['SUSP_SEX'] == 'F')]

vict = susp[(susp['VIC_SEX'] == 'M') | (susp['VIC_SEX'] == 'F')]

victcount = pd.crosstab(vict.SUSP_SEX, vict.VIC_SEX)

victprobs = pd.crosstab(vict.SUSP_SEX, vict.VIC_SEX) / len(vict)
victprobs
### END SOLUTION

### 3.11c Observations 
Based on what was calculated in 3.11a and 3.11b, state at least important observations.
##### BEGIN SOLUTION
In the matrix for 3.11a, we can make the assumption that the most frequent calls are incidents involving suspects aged 25-44 and vicitims aged 25-44. We can also infer that adults aged <18 and 65+ are the least likely to become suspects or victims in a call. 

As for the matrix in 3.11b, we can see that incidents involving a male suspect and female victim are the most frequent in the number of calls. We can also add that males are more likely to be suspects and victims than females.
##### END SOLUTION

### 3.12 - Visualization of crimes on a Map of NYC
Finally we attempt to visualize the crimes committed in NYC on a Map. For this activity, use all data, not just the First set,  NYPD_Complaint_Data_Historic_1. 
First we need to install some mapping software. RSun the cell below to install folium package for mapping software.

In [ ]:
!pip install --upgrade folium

In [ ]:
### Plot the maps for BURGLARY in NYC
### if it takes too much time or map does not show up, try plotting a subset of the data set of for a specific crime

### BEGIN SOLUTION
import folium
map = folium.Map(location=[40.693943, -73.985880], default_zoom_start=15)

nyburglary = calls[calls['OFNS_DESC'] == 'BURGLARY']

locations = nyburglary[['Latitude', 'Longitude']].dropna()
locationlist = locations.values.tolist()

for point in range(0, 20):
    folium.Marker(locationlist[point]).add_to(map)
    
map


### END SOLUTION

 ## Congratulations !!!

<div class="alert alert-block alert-info">
<h2>Submission Instructions</h2> 
<b> Output:</b> Please **remove all output** from your notebook prior to submission<br>
<b> File Name:</b> Please name the file as your_section_your_netID_Lab3.ipynb (eg. 01_adg133_Lab3.ipynb<br>
<b> Submit To: </b> Canvas &rarr; Assignments &rarr; Lab3 <br>
<b>Warning:</b> Failure to follow directions may result in loss of points.<br>
</div>

@2022 A.D. Gunawardena. Many people contributed to this lab including TA Liqin Long (now at Google). Much credits go to Josh Hug, and Berkeley Data Science Group for their contributions to the original version. Please DO NOT share this lab and/or post them on public sites.